In [ ]:
#!pip install boxmot

In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from boxmot import BoTSORT
from pathlib import Path
frame_width, frame_height = 1280,720
model = YOLO("yolov8l.pt")



# Draw Bounding boxes for each detection

In [ ]:
def drawBoundingBox(detections,frame):
    for box,cls in zip(detections[0].boxes.xyxy,detections[0].boxes.cls):
        if cls == 0:
            x1,y1,x2,y2 = map(lambda x:int(x),box)
            start_point = (x1, y1)
            end_point = (x2, y2)
            cv2.rectangle(frame,start_point ,end_point , (0, 255, 0), 2)
    return frame

# Detection

In [ ]:
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
ret, frame = cap.read() 
# tracker = BoTSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False)
while(cap.isOpened()):
  
    detections = model.predict(frame,verbose=False)
    if ret == True: 
        for box,cls in zip(detections[0].boxes.xyxy,detections[0].boxes.cls):
            if cls == 0:
                x1,y1,x2,y2 = map(lambda x:int(x),box)
                start_point = (x1, y1)
                end_point = (x2, y2)
                cv2.rectangle(frame,start_point ,end_point , (0, 255, 0), 2)
        cv2.imshow('Frame', frame)   
        ret, frame = cap.read()         
        if cv2.waitKey(25) & 0xFF == ord('q'): 
            break
    else:
        break

cap.release() 
cv2.destroyAllWindows() 



In [ ]:
s = set()
s.add(1)

In [ ]:
import sys

In [ ]:
def calculate_color_based_on_id(id):
  colors = [[255, 0, 0], # Red
          [0, 255, 0], # Green
          [0, 0, 255], # Blue
          [255, 255, 0], # Yellow
          [255, 0, 255], # Magenta
          [0, 255, 255], # Cyan
          [128, 128, 128], # Gray
          [255, 255, 255], # White
          [0, 0, 0], # Black
          [192, 192, 192]]
  return colors[id % len(colors) ]

In [ ]:
calculate_color_based_on_id(13)

# BOTSORT TRACKER 

In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from boxmot import BoTSORT
from pathlib import Path
frame_width, frame_height = 1280,720
model = YOLO("yolov8l.pt")

def calculate_color_based_on_id(id):
  colors = [[255, 0, 0], # Red
          [0, 255, 0], # Green
          [0, 0, 255], # Blue
          [255, 255, 0], # Yellow
          [255, 0, 255], # Magenta
          [0, 255, 255], # Cyan
          [128, 128, 128], # Gray
          [255, 255, 255], # White
          [0, 0, 0], # Black
          [192, 192, 192]]
  return colors[id % len(colors) ]



cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
ret, frame = cap.read() 
tracker = BoTSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False)
thickness = 2
color = (0,0,255)
while(cap.isOpened()):
    id_set = set()
    detections = model.predict(frame,verbose=False)[0]
    results = []
    if not ret:
        break

    if np.array(detections.boxes.data.tolist()).ndim < 2:
        results = [[0, 0, 0, 0, 0.0922948837280273, 0]] 
    try:
        ts = tracker.update(np.array(detections.boxes.data.tolist()), frame)
        xyxys = ts[:,0:4].astype('int') # float64 to int
        ids = ts[:, 4].astype('int') # float64 to int 
        confs = ts[:, 5]
        clss = ts[:, 6]
        if ts.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):

                if cls != 0:
                    continue
                id_set.add(id)
                cv2.rectangle(
                    frame,
                    (xyxy[0], xyxy[1]),
                    (xyxy[2], xyxy[3]),
                    calculate_color_based_on_id(id),
                    thickness
                )
                cv2.putText(
                    frame,
                    f'id: {id}',
                    (xyxy[0], xyxy[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    calculate_color_based_on_id(id),
                    thickness
                )
            cv2.putText(
                    frame,
                    f'Count: {len(id_set)}',
                    (10, 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            cv2.imshow('Frame', frame)
    except Exception as e:
         print("Error:",e)
         pass
    ret, frame = cap.read()         
    if cv2.waitKey(25) & 0xFF == ord('q'): 
            break

cap.release() 
cv2.destroyAllWindows() 

In [1]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from boxmot import BoTSORT,StrongSORT
from pathlib import Path
frame_width, frame_height = 1280,720
model = YOLO("yolov8l.pt")

def calculate_color_based_on_id(id):
  colors = [[255, 0, 0], # Red
          [0, 255, 0], # Green
          [0, 0, 255], # Blue
          [255, 255, 0], # Yellow
          [255, 0, 255], # Magenta
          [0, 255, 255], # Cyan
          [128, 128, 128], # Gray
          [255, 255, 255], # White
          [0, 0, 0], # Black
          [192, 192, 192]]
  return colors[id % len(colors) ]



cap = cv2.VideoCapture("./lab.mp4")
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
ret, frame = cap.read() 
tracker = StrongSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False)
thickness = 2
color = (0,0,255)
while(cap.isOpened()):
    id_set = set()
    detections = model.predict(frame,verbose=False)[0]
    results = []
    if not ret:
        break

    if np.array(detections.boxes.data.tolist()).ndim < 2:
        results = [[0, 0, 0, 0, 0.0922948837280273, 0]] 
    try:
        ts = tracker.update(np.array(detections.boxes.data.tolist()), frame)
        xyxys = ts[:,0:4].astype('int') # float64 to int
        ids = ts[:, 4].astype('int') # float64 to int 
        confs = ts[:, 5]
        clss = ts[:, 6]
        if ts.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):

                if cls != 0:
                    continue
                id_set.add(id)
                cv2.rectangle(
                    frame,
                    (xyxy[0], xyxy[1]),
                    (xyxy[2], xyxy[3]),
                    calculate_color_based_on_id(id),
                    thickness
                )
                cv2.putText(
                    frame,
                    f'id: {id}',
                    (xyxy[0], xyxy[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    calculate_color_based_on_id(id),
                    thickness
                )
            cv2.putText(
                    frame,
                    f'Count: {len(id_set)}',
                    (10, 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            cv2.imshow('Frame', frame)
    except Exception as e:
         print("Error:",e)
         pass
    ret, frame = cap.read()         
    if cv2.waitKey(25) & 0xFF == ord('q'): 
            break

cap.release() 
cv2.destroyAllWindows() 

2023-11-03 12:20:51.067 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2023-11-03 12:20:51.068 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


Error: too many indices for array: array is 1-dimensional, but 2 were indexed


QObject::moveToThread: Current thread (0x5573814b6d50) is not the object's thread (0x5572db476f40).
Cannot move to target thread (0x5573814b6d50)

qt.qpa.plugin: Could not load the Qt platform plugin "wayland" in "/home/loki/Development/major-project/RealTime-Webcam/venv/lib/python3.11/site-packages/cv2/qt/plugins" even though it was found.
